<img src = "https://github.com/singlestore-labs/spaces-notebooks/blob/e551e274bb67bb1e5081131ee1150cdba713fc43/common/images/singlestore-jupyter.png?raw=true">

<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Movie Recommender Part 4</h1>
    </div>
</div>

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Select the database from the drop-down menu at the top of this notebook.</p>
    </div>
</div>

# Example Queries

## Compute Similarity Between a User and a Movie

This query computes the dot product between a specific user's factors and a specific movie's factors to determine the similarity score.

In [5]:
%%sql
SELECT
    u.id AS user_id,
    m.title,
    (u.factors <*> m.factors) AS similarity_score
FROM
    users u, movies m
WHERE
    u.id = 1
    AND m.id = 1240; -- The Terminator

user_id,title,similarity_score
1,"Terminator, The (1984)",-0.05929713696241379


## Find Top Movies for a User Based on Similarity

This query finds movies that are most similar to a particular user based on dot product similarity.

In [6]:
%%sql
SELECT
    m.title,
    m.genres,
    (u.factors <*> m.factors) AS similarity_score
FROM
    users u, movies m
WHERE
    u.id = 1
ORDER BY
    similarity_score DESC
LIMIT 10;

title,genres,similarity_score
Airport (1970),Drama,0.5594869256019592
Center Stage (2000),Drama,0.49209100008010864
"Dark Half, The (1993)",Horror|Mystery,0.4820263087749481
Christmas Vacation (1989),Comedy,0.4356957674026489
Beauty and the Beast (1991),Animation|Children's|Musical,0.4331190288066864
Caddyshack (1980),Comedy,0.4262603521347046
"Love Bug, The (1969)",Children's|Comedy,0.42389824986457825
Willy Wonka and the Chocolate Factory (1971),Adventure|Children's|Comedy|Fantasy,0.4190407991409302
Awakenings (1990),Drama,0.4100484251976013
Hush (1998),Thriller,0.40862804651260376


<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Select the database from the drop-down menu at the top of this notebook. It updates the <b>connection_url</b> which is used by SQLAlchemy to make connections to the selected database.</p>
    </div>
</div>

In [7]:
from sqlalchemy import *

db_connection = create_engine(connection_url)

In [8]:
import pandas as pd

df = pd.read_sql("""
SELECT
    m.title,
    m.poster,
    (u.factors <*> m.factors) AS similarity_score
FROM
    users u, movies m
WHERE
    u.id = 1
ORDER BY
    similarity_score DESC
LIMIT 10;
""", con = db_connection)

In [9]:
# Display movie recommendations with posters
from IPython.display import display, HTML

# Function to generate HTML img tag for each row
def display_image(url):
    return f'<img src = "{url}" width = "100">'

# Generate HTML content for table with outline and text/images for posters
html_table = []
html_table.append('<table style = "border-collapse: collapse; width: 100%; border: 1px solid #ddd;">')
html_table.append('<tr style = "text-align: left; border-bottom: 1px solid #ddd;">')
html_table.append('<th style = "padding: 10px; border-right: 1px solid #ddd;">Title</th>')
html_table.append('<th style = "padding: 10px;">Poster</th>')
html_table.append('</tr>')

# Iterate over DataFrame rows to populate table rows
for i in range(len(df)):
    html_table.append('<tr>')
    html_table.append(f'<td style = "padding: 10px; border-right: 1px solid #ddd;">{df["title"].iloc[i]}</td>')
    html_table.append(f'<td style = "padding: 10px;">{display_image(df["poster"].iloc[i])}</td>')
    html_table.append('</tr>')

html_table.append('</table>')

display(HTML(''.join(html_table)))

Title,Poster
Airport (1970),
Center Stage (2000),
"Dark Half, The (1993)",
Christmas Vacation (1989),
Beauty and the Beast (1991),
Caddyshack (1980),
"Love Bug, The (1969)",
Willy Wonka and the Chocolate Factory (1971),
Awakenings (1990),
Hush (1998),


## Calculate Similarity Matrix Between Users and Movies

This query calculates a similarity matrix where each cell represents the similarity score between a user and a movie.

In [10]:
%%sql
SELECT
    u.id AS user_id,
    m.title,
    (u.factors <*> m.factors) AS similarity_score
FROM
    users u, movies m
ORDER BY similarity_score DESC
LIMIT 10;

user_id,title,similarity_score
4539,H.O.T.S. (1979),3.218724489212036
4539,Porky's Revenge (1985),3.056617259979248
4539,Porky's II: The Next Day (1983),3.048598289489746
5326,Bride of Chucky (1998),2.6750497817993164
4539,Striptease (1996),2.637443780899048
5597,Children of the Corn (1984),2.624882221221924
4028,"Big Hit, The (1998)",2.6105029582977295
3610,"Waterboy, The (1998)",2.575476884841919
5597,Friday the 13th Part 2 (1981),2.5469560623168945
3537,Nutty Professor II: The Klumps (2000),2.5355920791625977


## Recommend Top Movies for Each User

This query recommends top movies for each user based on their similarity scores with movies.

In [11]:
%%sql
SELECT
    u.id AS user_id,
    m.title,
    (u.factors <*> m.factors) AS similarity_score
FROM
    users u, movies m
ORDER BY
    u.id, similarity_score DESC
LIMIT 10;

user_id,title,similarity_score
1,Airport (1970),0.5594869256019592
1,Center Stage (2000),0.49209100008010864
1,"Dark Half, The (1993)",0.4820263087749481
1,Christmas Vacation (1989),0.4356957674026489
1,Beauty and the Beast (1991),0.4331190288066864
1,Caddyshack (1980),0.4262603521347046
1,"Love Bug, The (1969)",0.42389824986457825
1,Willy Wonka and the Chocolate Factory (1971),0.4190407991409302
1,Awakenings (1990),0.4100484251976013
1,Hush (1998),0.40862804651260376


## Find Users Similar to Toy Story

Identify users who are most similar to **Toy Story** in terms of preferences and characteristics.

In [12]:
%%sql
SELECT
    u.gender,
    u.age,
    u.occupation,
    u.zip_code,
    (m.factors <*> u.factors) AS similarity_score
FROM
    movies m, users u
WHERE
    m.id = 1 -- Toy Story
ORDER BY similarity_score DESC
LIMIT 10;

gender,age,occupation,zip_code,similarity_score
F,18,college/grad student,5849574,1.1796190738677979
F,18,college/grad student,44243,0.8611005544662476
M,35,academic/educator,49307,0.8208982348442078
F,35,programmer,91104,0.8108484148979187
F,35,clerical/admin,01545,0.7916239500045776
M,35,other,33174,0.7710853219032288
M,18,other,95037,0.769222617149353
M,56,academic/educator,30345,0.7633796334266663
M,18,college/grad student,06511,0.7617809176445007
F,50,tradesman/craftsman,20706,0.7557101845741272


## Find Movies Similar to Toy Story

Identify movies who are most similar to **Toy Story**.

In [13]:
%%sql
SELECT
    m.title,
    (m.factors <*> qv.factors) AS similarity_score
FROM
    movies m,
    (SELECT factors AS factors FROM movies WHERE id = 1) AS qv -- Toy Story
ORDER BY
    similarity_score DESC
LIMIT 10;

title,similarity_score
Toy Story (1995),1.4211541414260864
Toy Story 2 (1999),1.2376554012298584
Aladdin (1992),1.1187670230865479
Beauty and the Beast (1991),1.0759620666503906
Babe (1995),1.0588488578796387
"Little Mermaid, The (1989)",1.0443871021270752
"Bug's Life, A (1998)",0.8562913537025452
Babe: Pig in the City (1998),0.8382574319839478
Lady and the Tramp (1955),0.8161893486976624
"Lion King, The (1994)",0.7789281606674194


## Top Movies Predicted Ratings

Predict the top-rated movies based on demographic attributes for a new user.

In [14]:
%%sql
SELECT
    m.title,
    m.genres,
    (m.factors <*> u.factors) AS predicted_rating
FROM
    movies m, users u
WHERE
    u.gender = 'F'
    AND u.age = 18
ORDER BY
    predicted_rating DESC
LIMIT 10;

title,genres,predicted_rating
Children of the Corn (1984),Horror|Thriller,2.624882221221924
Friday the 13th Part 2 (1981),Horror,2.5469560623168945
Pet Sematary II (1992),Horror,2.4629597663879395
Howling II: Your Sister Is a Werewolf (1985),Horror,2.2804813385009766
Fear and Loathing in Las Vegas (1998),Comedy|Drama,2.264206647872925
Bride of Chucky (1998),Horror|Thriller,2.146353006362915
Creepshow 2 (1987),Horror,2.1201846599578857
Natural Born Killers (1994),Action|Thriller,2.04565691947937
Halloween: The Curse of Michael Myers (1995),Horror|Thriller,1.957294225692749
Blue Hawaii (1961),Comedy|Musical,1.955091953277588


## Top Average Ratings by Genre

Calculate the average rating for movies in each genre and list the genres with the highest average ratings, along with the number of ratings received.

In [15]:
%%sql
SELECT
    genres,
    AVG(rating) AS avg_rating,
    COUNT(*) AS num_ratings
FROM
    movies m
JOIN
    ratings r ON m.id = r.movie_id
GROUP BY
    genres
ORDER BY
    avg_rating DESC
LIMIT 10;

genres,avg_rating,num_ratings
Animation|Comedy|Thriller,4.473837209302325,688
Sci-Fi|War,4.4498902706656915,1367
Animation,4.394335511982571,459
Film-Noir|Mystery,4.367424242424242,1584
Adventure|War,4.34610705596107,1644
Film-Noir|Romance|Thriller,4.29438202247191,445
Action|Adventure|Drama|Sci-Fi|War,4.292976588628763,2990
Film-Noir|Sci-Fi,4.273333333333333,1800
Crime|Film-Noir,4.264129181084199,867
Film-Noir,4.258104738154613,802


## User Activity Overview

An overview of user activity, including the count and average rating of movies rated by users grouped by gender and age.

In [16]:
%%sql
SELECT
    u.gender,
    u.age,
    COUNT(r.rating) AS num_ratings,
    AVG(r.rating) AS avg_rating
FROM
    users u
LEFT JOIN
    ratings r ON u.id = r.user_id
GROUP BY
    u.gender,
    u.age
ORDER BY
    u.gender,
    u.age;

gender,age,num_ratings,avg_rating
F,1,8827,3.616290925569276
F,18,45427,3.4531446056310124
F,25,91340,3.6067002408583315
F,35,49473,3.6596527398783176
F,45,24110,3.663044379925342
F,50,18064,3.7971102745792735
F,56,9199,3.915534297206218
M,1,18384,3.5174608355091386
M,18,138109,3.525476254262937
M,25,304216,3.52678031398743


## Most Active Users

The most active users based on the number of movie ratings they have provided, along with their demographic information.

In [17]:
%%sql
SELECT
    u.id,
    u.gender,
    u.age,
    COUNT(r.rating) AS num_ratings
FROM
    users u
LEFT JOIN
    ratings r ON u.id = r.user_id
GROUP BY
    u.id,
    u.gender,
    u.age
ORDER BY
    num_ratings DESC
LIMIT 10;

id,gender,age,num_ratings
4169,M,50,2314
1680,M,25,1850
4277,M,35,1743
1941,M,35,1595
1181,M,35,1521
889,M,45,1518
3618,M,56,1344
2063,M,25,1323
1150,F,25,1302
1015,M,35,1286


## Average Rating Over Time

Query analyzes average movie ratings over different years based on rating timestamps, providing insights into rating trends over time.

In [18]:
%%sql
SELECT
    YEAR(FROM_UNIXTIME(r.timestamp)) AS rating_year,
    AVG(r.rating) AS avg_rating
FROM
    ratings r
GROUP BY
    rating_year
ORDER BY
    rating_year;

rating_year,avg_rating
2000,3.5903916742285498
2001,3.5122542537247643
2002,3.458828911253431
2003,3.486559139784946
